In [ ]:
# Download COCO dataset with support for resume from breakpoint if partial file already exists.

import os
import requests

def download_file(url, local_filename):
    # Check if file already exists and determine its size
    if os.path.exists(local_filename):
        # Get the size of the already downloaded file
        downloaded_size = os.path.getsize(local_filename)
    else:
        downloaded_size = 0

    # If the file exists and has content, set the Range header to resume download
    headers = {}
    if downloaded_size:
        headers = {'Range': f'bytes={downloaded_size}-'}

    with requests.get(url, stream=True, headers=headers) as r:
        # Check if the server indicates the file is already fully downloaded
        if downloaded_size and r.status_code == 416:
            print(f"{local_filename} already fully downloaded.")
            return local_filename

        r.raise_for_status()

        # Open file in append binary mode if resuming, otherwise write binary mode
        mode = 'ab' if downloaded_size else 'wb'
        with open(local_filename, mode) as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:  # Filter out keep-alive chunks
                    f.write(chunk)
                    downloaded_size += len(chunk)
                    print(f"Downloaded {downloaded_size} bytes of {local_filename}")

    return local_filename

# Download COCO dataset
urls = [
    'http://images.cocodataset.org/zips/train2017.zip',
    'http://images.cocodataset.org/zips/val2017.zip',
    'http://images.cocodataset.org/zips/test2017.zip',
    'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
]

for url in urls:
    filename = os.path.join(os.getcwd(), os.path.basename(url))
    print(f'Downloading {filename}...')
    download_file(url, filename)
    print(f'{filename} downloaded successfully.')
